# EFFECTIVE MODERN C++

<hr>

## ITEMS

- **Deducing types**<br>
**Item 1**: Understand template type deduction.<br>
**Item 2**: Understand `auto` type deduction.<br>
**Item 3**: Understand `decltype`.<br>
**Item 4**: Know how to view deduced types.<br>


- **auto**<br>
**Item 5**: Prefer `auto` to explicit type declarations.<br>
**Item 6**: Use the `explicitly typed initializer idiom` when `auto` deduces undesired types.<br>


- **Moving to modern C++**<br>
**Item 7**: Distinguish between `()` and `{}` when creating objects.<br>
**Item 8**: Prefer `nullptr` to `0` and `NULL`.<br>
**Item 9**: Prefer `alias` declarations to `typedefs`.<br>
**Item 10**: Prefer scoped enums to unscoped enums.<br>
**Item 11**: Prefer deleted functions to private undefined ones.<br>
**Item 12**: Declare overriding functions `override`.<br>
**Item 13**: Prefer `const_iterators` to `iterators`.<br>
**Item 14**: Declare functions `noexcept` if they won't emit exceptions.<br>
**Item 15**: Use `constexpr` whenever possible.<br>
**Item 16**: Make `const` member functions thread safe.<br>
**Item 17**: Understand special member function generation.<br>


- **Smart pointers**<br>
**Item 18**: Use `std::unique_ptr` for exclusive-ownership resource management.<br>
**Item 19**: Use `std::shared_ptr` for shared-ownership resource management.<br>
**Item 20**: Use `std::weak_ptr` for `std::shared_ptr`-like pointers that can dangle.<br>
**Item 21**: Prefer `std::make_unique` and `std::make_shared` to direct use of `new`.<br>
**Item 22**: When using the `Pimpl Idiom`, define special member functions in the implementation file.


- **Rvalue references, move semantics, and perfect forwarding**<br>
**Item 23**: Understand `std::move` and `std::forward`.


<hr>

## PREFACE

* `Rvalues` indicate object eligible for move operations, while `lvalues` generally don't.
* `Rvalues` correspond to objects you can refer to, either by name or by following a pointer or `lvalue` reference.

A useful heuristic to determine whether an expression is an `lvalue` is to ask if you can take its address. If you can, it typically is.

<pre><code><font color="blue">public:</font>
    Widget(Widget<font color="blue">&&</font> rhs);                       <font color="green">// rhs (right hand side) is an rvalue</font>
</code></pre>

Copies of `rvalues` are generally move constructed, while copies of `lvalues` are usally copy constructed.

<pre><code>
<font color="blue">void</font> someFunc(Widget w);
someFunc(<font color="green">std::move</font>(wid))                        <font color="green">// rvalue semantic</font>
</code></pre>

* The arguments are used to initialize the function's parameters.
* The distinction between arguments and parameters is important, because `parameters` are lvalues, but the `arguments` with which they are initialized may be rvalues or lvalues.

**perfect forwarding** - an argument passed to a function is passed to a second function such that the original argument's rvalueness or lvalueness is preserved

**function object** - an object that acts like a function; an object of type supporting an `operator()` member function

**callable objects** - anything that can be invoked using the syntax of a non-member function call; this covers `operator()`, functions, C-like function pointers, member function pointers

**closures** - function objects created through lambda expressions


* `Declarations` introduce names and types without giving details.
* `Definitions` provide the storage locations or implementation details.

**function's signature** - a part of its declaration that specifies parameter and return types

**undefined behavior** - runtime behavior is unpredictable

**`std::move`** - treat argument as a temporary object (rvalue); casting rvalue to lvalue is allowed

**`std::forward`** - cast a templated function parameter (inside the function) to the value category (lvalue or rvalue); this allows performing a perfect forwarding; casting rvalue to lvalue is prohibited

<hr>

## DEDUCING TYPES FOR FUNCTION TEMPLATES
<pre><code><font color="blue">template</font><<font color="blue">typename</font> T> <font color="blue">void</font> f(ParamType param); 
f(expr);
</code></pre>

* `ParamType` is a pointer or reference type, but not a universal reference - `T&`
    * If expr's type is a reference, ignore the reference part.
    * Then pattern-match expr's type against `ParamType` to determinate `T`.
* `ParamType` is a universal reference - `T&&`
    * If expr is an `lvalue`, both `T` and `ParamType` are deduced to be lvalue references.
    * If expr is an `rvalue`, the 'normal' rules (from above) apply.
* `ParamType` is neither a pointer nor a reference - `T`
    * If expr's type is a reference, ignore the reference part.
    * If, expr is `const` or `volatile` ignore that, too.
* `ParamType` is an array or a function argument (arr- and func-to-ptr decay)


Examples:

<pre><code><font color="blue">int</font> x = 27; 
<font color="blue">const int</font> cx = x; 
<font color="blue">volatile const int</font>& vx = x;
<font color="blue">const char</font>* <font color="blue">const</font> ptr = "Fun with pointers";
<font color="blue">const char</font> name[] = "J. P. Briggs";
<font color="blue">void</font> someFunc(<font color="blue">int</font>, <font color="blue">double</font>);

if ParamType:=<font color="green">T&</font>  calling <font color="green">f(cx);</font>       then T:=<font color="blue">const int</font>            and param:=<font color="blue">const int &</font>
if ParamType:=<font color="green">T&&</font> calling <font color="green">f(x);</font>        then T:=<font color="blue">int&</font>                 and param:=<font color="blue">int&</font>
if ParamType:=<font color="green">T&&</font> calling <font color="green">f(27);</font>       then T:=<font color="blue">int</font>                  and param:=<font color="blue">int&&</font>
if ParamType:=<font color="green">T</font>   calling <font color="green">f(vx);</font>       then T:=<font color="blue">int</font>                  and param:=<font color="blue">int</font>
if ParamType:=<font color="green">T</font>   calling <font color="green">f(ptr);</font>      then T:=<font color="blue">const char* const</font>    and param:=<font color="blue">const char*</font>
if ParamType:=<font color="green">T</font>   calling <font color="green">f(name);</font>     then T:=<font color="blue">const char*</font>          and param:=<font color="blue">const char*</font>
if ParamType:=<font color="green">T&</font>  calling <font color="green">f(name);</font>     then T:=<font color="blue">const char[13]</font>       and param:=<font color="blue">const char(&)[13]</font>
if ParamType:=<font color="green">T&</font>  calling <font color="green">f(someFunc);</font> then T:=<font color="blue">void (&)(int,double)</font> and param:=<font color="blue">ref-to-func</font>
</code></pre>

**constexpr** - makes a function declaration result available during compilation

**noexcept** - help compilers to generate a better code

Trailing return type syntax example:

<pre><code><font color="blue">template</font><<font color="blue">typename</font> Container, <font color="blue">typename</font> Index>
<font color="blue">auto</font> authAndAccess(Container& c, Index i) -> <font color="blue">decltype</font>(<font color="green">std:forward</font><Container>(c)[i]) { 
    authenticateUser(); 
    <font color="blue">return</font> c[i]; 
    <font color="green">// C++11</font>
}

<font color="blue">template</font><<font color="blue">typename</font> Container, <font color="blue">typename</font> Index>
<font color="blue">decltype</font>(<font color="blue">auto</font>) authAndAccess(Container&& c, Index i) {
    authenticateUser();
    <font color="blue">return</font> <font color="green">std::forward</font><Container>(c)[i];
    <font color="green">// C++14</font>
}
</code></pre>


<hr>

## AUTO KEYWORD

**function object** - `std::function` template is a generalization of a function that can point to any callable object (anything that can be invoked as a function); this approach is generally bigger and slower compared with `auto` approach (closure hold in memory, restricted inlining)


<pre><code>std::vector<<font color="blue">int</font>> v;
std::vector<<font color="blue">int</font>>::size_type           // official type: unsigned integral type
<font color="blue">unsigned</font> sz = v.size();               // on 64-bit Windows: 32 bits
<font color="blue">auto</font> sz = v.size();                   // on 64-bit Windows: 64 bits
</code></pre>


<pre><code>std::unordered_map<<font color="blue">std::string</font>, <font color="blue">int</font>> m;

<font color="blue">for</font> (<font color="blue">const std::pair</font><<font color="blue">std::string</font>, <font color="blue">int</font>>& p : m) {
    <font color="green">// key is const!
    // type of the pair in the hash table is std::pair&lt;const std::string, int&gt;
    // compiler will do a convertion to std::pair&lt;string, int&gt; for a temporary object
    // explicit type specification leads to implicit convertions</font>
}
</code></pre>


<pre><code>std::vector<<font color="blue">bool</font>> features(<font color="blue">const</font> Widget& w);
Widget w;

<font color="blue">bool</font> highPriority = features(w)[5];                           // OK
<font color="blue">auto</font> highPriority = features(w)[5];                           // undefined behavior
<font color="blue">auto</font> highPriority = <font color="blue">static_cast</font><<font color="blue">bool</font>>(features(w)[5]);        // explicitly typed initializer idiom

processWidget(w, highPriority);
<font color="green">// std::vector<T> holds operator[] that is supposed to return a T&
// operator[] returns a reference to an element of the container, except bool
// for bools operator[] return std::vector<bool>::reference, a nested class (proxy class)
// std::vector<bool> is specified to represent its bools in packed form, one bit per bool
// yet, C++ forbids references to bits
// not being able to return bool&, operator[] returns an object that acts like a bool&
// to make this work we need an implicit conversion to bool, not to bool&
// auto deduced type depends on the ::reference implementation (i.e. ptr-to-word + offset)
// the call to features returns a temporary std::vector<bool> object temp
// invoked temp->operator[] returns ::reference containing a ptr-to-word plus offset[5]
// highPriority is a copy of a pointer to a word in temp with its offset
// at the end of the statement temp is destroyed because is a temporary object
// this means that highPriority contains a dangling pointer!</font>
</code></pre>

**proxy class**: a class that exists for the purpose of emulating and augmenting the behavior of some other type
* `std::vector<bool>::reference` offers the illusion that `operator[]` returns a reference to a bit
* STL smart pointers types graft resource management onto raw pointers
* apparent `Proxies` examples: `std::shared_ptr`, `std::unique_ptr`
* invisible `Proxies` examples: `std::vector<bool>::reference`, `std::bitset::reference`, classes from STL using expression templates


<pre><code><font color="green">// expression template
// originally developed to improve the efficiency of numeric code</font>
Matrix sum = m1 + m2 + m3 + m4;

<font color="blue">auto</font> sum = <font color="blue">static_cast</font>&lt;Matrix&gt;(m1 + m2 + m3 + m4);
<font color="green">// explicitly typed initializer idiom
// operator+ for Matrix should return a proxy for the result
// for two Matrix objects the result should be Sum&lt;Matrix, Matrix&gt;
// we need to provide an implicit conversion from the proxy class to Matrix
// the entire initialization expression: SumSum&lt;Sum&lt;Matrix, Matrix&gt;, Matrix&gt;, Matrix&gt;

// general rule: invisible proxy classes don't play well with auto keyword
// proxy classes are often designed to live for a single statement
// creating variables of those types tends to violate fundamental library design assumptions
// how to recognize when proxy objects are in use?
// function signatures usually reflect their existence (they're typically returned from those)
// also, pay careful attention to the interfaces you're using</font></code></pre>

<pre><code><font color="green">// explicitly typed initializer idiom</font>
<font color="blue">double</font> calcEpsilon();
<font color="blue">float</font> ep = calcEpsilon();                    <font color="green">          // may be or not deliberate, intend not clear</font>
<font color="blue">auto</font> ep = <font color="blue">static_cast</font><<font color="blue">float</font>>(calcEpsilon()); <font color="green">          // deliberate type reduction</font></code></pre>

<hr>

## DISTINGUISH BETWEEN PARENTHESES AND BRACES
**uniform initialization**: a single initialization syntax that can, at least in concept, be used anywhere and express everything
* `'uniform initialization'` is an idea, `'braced initialization'` is a syntactic construct
* specyfing the initial contents of a container is easy
* braces can be used to specify default initialization values for non-static data members
* it prohibits implicit narrowing conversions among built-in types
* it is immune to C++'s `most vexing parse`
* if one or more constructors declare a parameter of type `std::initializer_list`, calls using the braced iniitialization syntax **strongly** prefer the overloads taking this type

**most vexing parse**: a side effect of C++'s rule that anything that can be parsed as a declaration must be interpreted as one; usually afflicts situations when you want to default-construct an object, but inadvertently declaring a function instead

<pre><code>Widget w1;                                <font color="green">// call default constructor</font>
Widget w2 = w1;                           <font color="green">// calls copy ctor</font>
w1 = w2;                                  <font color="green">// call copy operator=</font>
std::vector<<font color="blue">int</font>> v{ 1, 3, 5 };            <font color="green">// initial content is in braces</font>
std::atomic<<font color="blue">int</font>> ai1{ 0 };                <font color="green">// assignment, where '=' is not allowed</font>
<font color="blue">double</font> x, y, z;
<font color="blue">int</font> sum1{ x + y + z };                    <font color="green">// ERROR, double not expressible as int</font>
<font color="blue">int</font> sum2(x + y + z);                      <font color="green">// OK, expression truncated to int</font>
<font color="blue">int</font> sum3 = x + y + z;                     <font color="green">// ditto</font>
Widget w3(10);                            <font color="green">// call Widget ctor with argument 10</font>
Widget w4();                              <font color="green">// declares a function w2 that returns Widget</font>
Widget w5{};                              <font color="green">// calls Widget ctor with no args</font>
</code></pre>

<pre><code><font color="blue">class</font> Widget {
<font color="blue">public</font>:
    Widget(<font color="blue">int</font> i, <font color="blue">bool</font> b);
    Widget(<font color="blue">int</font> i, <font color="blue">double</font> d);
    Widget(std::initializer_list<<font color="blue">long double</font>> il);
    <font color="blue">operator</font> float() <font color="blue">const</font>;                             <font color="green">// conversion operator</font>
};

Widget w1(10, true);                     <font color="green">// first ctor</font>
Widget w3(10, 5.0);                      <font color="green">// second ctor</font>
Widget w2{10, true};                     <font color="green">// third ctor, conversion to long double</font>
Widget w4{10, 5.0};                      <font color="green">// ditto</font>
Widget w5(w4);                           <font color="green">// copy ctor</font>
Widget w7(std::move(w4));                <font color="green">// move ctor</font>
Widget w6{w4};                           <font color="green">// third ctor, w4 -> float -> long double</font>
Widget w8{std::move(w4)};                <font color="green">// ditto</font>
Widget w9{};                             <font color="green">// edge case, the rule is default construction</font>
Widget w10({});                          <font color="green">// third ctor, empty list</font>
std::vector<int> v1(10, 20);                  <font color="green">// 10 elements, each of value 20</font>
std::vector<int> v2{10, 20};                  <font color="green">// 2 elements, values 10 and 20</font>
                                         <font color="green">// is now viewed as an error in the interface design</font>
                                         <font color="green">// il overloads overshadows others overloads</font>
                                         <font color="green">// in auto type deduction {} is a std:initializer_list</font>
</code></pre>

<pre><code><font color="blue">template</font><<font color="blue">typename</font> T, <font color="blue">typename</font>... Ts>
<font color="blue">void</font> doSomeWork(Ts&&... params) {
    <font color="green">// T - type of objects to create
    // Ts - types of arguments to use
    // we either use parens or braces for parsing</font>
    <font color="blue">T</font> localObject(std::forward<Ts>(params)...);
    <font color="blue">T</font> localObject<font color="red">{</font>std::forward<Ts>(params)...<font color="red">}</font>;
}

std::vector<<font color="blue">int</font>> v;
doSomeWork&lt;std::vector<<font color="blue">int</font>>>(10, 20);
<font color="green">// for parens, v has 10 elements; for braces, v has 2 elements
// which is correct? only the caller can know!</font>
</code></pre>

<hr>

## DIFFERENCES BETWEEN 0, NULL, NULLPTR
* literal `0` is an `int`, not a `pointer`
* if `0` is used in a context where only a pointer can be used, it'll be interpreted as `null pointer`, which is a fallback position
* the same is true for `NULL`, yet it is allowed to give it an `integral` type other than `int`, especially `long`
* a guideline for C++98 is to avoid overloading on pointer and integral types because of function overload resolution surprises

<pre><code><font color="blue">void</font> f(<font color="blue">int</font>);
<font color="blue">void</font> f(<font color="blue">bool</font>);
<font color="blue">void</font> f(<font color="blue">void</font> *);

f(0);               <font color="green">// calls f(int)</font>
f(<font color="blue">NULL</font>);            <font color="green">// may not compile, otherwise calls f(int)
                    // if NULL is defined to be 0L than the call is ambiguous
                    // conversion long -> int, long -> bool, 0L -> void* are equally good</font>
</code></pre>

**`nullptr`**: a type that can be viewed as a pointer of all types
* `nullptr` does not have integral type nor pointer type
* `nullptr` type is `std::nullptr_t`
* `std::nullptr_t` is defined to be the type of `nullptr`
* `std::nullptr_t` implicitly converts to all raw pointer types
* overloaded function with `nullptr` argument always calls the `void*` overload
* preferred to use in templates

<hr>

## ALIASES AND TYPEDEFS
* aliases are possibly easier with function pointers
* alias declarations may be templatized (alias templates)

<pre><code><font color="blue">typedef</font> <font color="teal">std::unique_ptr</font>&lt;std::unordered_map&lt;<font color="teal">std::string</font>, <font color="teal">std::string</font>&gt;&gt; UPtrMapSS;
<font color="blue">using</font> UPtrMapSS = <font color="teal">std::unique_ptr</font>&lt;std::unordered_map&lt;<font color="teal">std::string</font>, <font color="teal">std::string</font>&gt;&gt;;

<font color="blue">typedef void</font> (*FP)(<font color="blue">int</font>, <font color="blue">const</font> std::string &);
<font color="blue">using</font> FP = <font color="blue">void</font> (*)(<font color="blue">int</font>, <font color="blue">const</font> std::string &);
</code></pre>

<pre><code><font color="blue">template</font>&lt;<font color="blue">typename</font> T>
<font color="blue">using</font> MyAllocList = <font color="teal">std::list</font>&lt;T, MyAlloc&lt;T&gt;&gt;;
<font color="green">// MyAllocList&lt;Widget&gt; lw;</font>

<font color="blue">template</font>&lt;<font color="blue">typename</font> T>
<font color="blue">struct</font> MyAllocList {
    <font color="blue">typedef</font> <font color="teal">std::list</font>&lt;T, MyAllocList&lt;T&gt;&gt; type;
};
<font color="green">// MyAllocList&lt;Widget&gt;::type lw;</font>
</code></pre>

<pre><code><font color="blue">template</font>&lt;<font color="blue">typename</font> T>
<font color="blue">class</font> Widget {
<font color="blue">private</font>:
    <font color="blue">typename</font> <font color="teal">MyAllocList</font>&lt;T&gt;::type list;
    <font color="green">// typedef type inside a template must be preceded with typename
    // nested MyAllocList&lt;T&gt;::type is a dependent type
    // ::type may not name a type (there might be a specialization of MyAllocList)</font>
};

<font color="blue">template</font>&lt;<font color="blue">typename</font> T> <font color="blue">using</font> MyAllocList = <font color="teal">std::list</font>&lt;T, MyAlloc&lt;T&gt;&gt;;
<font color="blue">template</font>&lt;<font color="blue">typename</font> T> <font color="blue">class</font> Widget {
<font color="blue">private</font>:
    <font color="teal">MyAllocList</font>&lt;T&gt; list;
    <font color="green">// alias template; typename is neither required nor permitted
    // MyAllocList must name a type, thus MyAllocList&lt;T&gt; is a non-dependent type</font>
};

<font color="blue">template</font>&lt;&gt; <font color="blue">class</font> MyAllocList&lt;<font color="teal">Wine</font>&gt; {
<font color="blue">private</font>:
    <font color="blue">enum class</font> WineType { White, Red, Rose };
    <font color="teal">WineType</font> type;
    <font color="green">// this a MyAllocList template specialization for T=Wine
    // type is a data member, MyAllocList&lt;T&gt;::type inside Widget will refer to it</font>
};
</code></pre>

In **template metaprogramming (TMP)** sometimes there is a need to take template type parameters and create a revised types from them. For example, we want to `T=const std::string` & turn into `std::string`. In C++11+ this type of transformations are available as templates and are called `type traits`. Type traits are implemented as nested `typedefs` inside templatized `structs` (reasons for this are historical, Standarization Committee first choose was typedef).

<pre><code>
std::<font color="teal">remove_const</font><T>::type                         <font color="green">// C++11, const T -> T</font>
std::<font color="teal">remove_const_t</font><T>                             <font color="green">// C++14</font>
std::<font color="teal">remove_reference</font><T>::type                     <font color="green">// T&/T&& -> T</font>
std::<font color="teal">remove_reference_t</font><T>
std::<font color="teal">add_lvalue_reference</font><T>::type                 <font color="green">// T -> T&</font>
std::<font color="teal">add_lvalue_reference_t</font><T>
std::<font color="teal">transformation</font><T>::type                       <font color="green">// C++11 common form</font>
std::<font color="teal">transformation_t</font><T>                           <font color="green">// C++14 alias template</font>
</code></pre>

<hr>

## SCOPED AND UNSCOPED ENUMS
* `enum classes` reduce namespace pollution by scoping their names (those in braces)
* `enum classes` are strongly typed (underlying type is `int`, usually `std::size_t`), whereas `unscoped enums` implicitly converts to integral types (usually `float` or `char`)
* `scoped enums` may be forward-declared (no need for enumerators in braces)

<pre><code>
<font color="blue">enum</font> Color { black, white, red };             <font color="green">// 4 names in the same scope, C++98</font>
<font color="blue">auto</font> white = false;                           <font color="green">// ERR, white already declared</font>

<font color="blue">enum class</font> Color { black, white, red };       <font color="green">// 3 names scoped to Color, C++11</font>
<font color="blue">auto</font> white = false;                           <font color="green">// fine</font>
<font color="teal">Color</font> c = white;                              <font color="green">// ERR, white is not declared</font>
<font color="blue">auto</font> c = <font color="teal">Color</font>::white;                        <font color="green">// fine</font>

<font color="blue">auto</font> comparison = c < 14.5;                   <font color="green">// OK, for unscoped enum</font>
<font color="blue">enum class</font> Status;                            <font color="green">// forward-declaration</font>

<font color="blue">enum class</font> Status : <font color="teal">std::uint32_t</font>;            <font color="green">// specyfing underlying type</font>
<font color="blue">enum</font> Color : <font color="teal">std::uint8_t</font>;                    <font color="green">// either at declaration or definition</font>
</code></pre>

<pre><code><font color="blue">template</font>&lt;<font color="blue">typename</font> E>
<font color="blue">constexpr auto</font> toUType(E enumerator) <font color="blue">noexcept</font> {
    <font color="green">// C++14</font>
    <font color="blue">return static_cast</font>&lt;std::<font color="teal">underlying_type_t</font>&lt;E&gt;&gt;(enumerator);
}

<font color="blue">enum class</font> UserInfoFields { uiName, uiEmail, uiReputation };
<font color="blue">using</font> UserInfo = std::<font color="teal">tuple</font>&lt;std::<font color="teal">string</font>, std::<font color="teal">string</font>, std::<font color="teal">size_t</font>&gt;;
<font color="teal">UserInfo</font> uInfo;
<font color="blue">auto</font> val = std::<font color="teal">get</font>&lt;toUType(<font color="teal">UserInfoFields</font>::uiEmail)&gt;(uInfo);
</code></pre>

<hr>

## DELETED FUNCTIONS VS PRIVATE FUNCTIONS
* making new functions `public` resulting in better error messages (private complains etc.)
* any function may be deleted, while only member functions may be private
* deleted functions can't be used
* deleted functions are taken into consideration during `overload resolution`
* deleted functions prevent use of a template instantiations that should be disabled

<pre><code><font color="blue">template</font>&lt;<font color="blue">class</font> charT, <font color="blue">class</font> traits = <font color="teal">char_traits</font>&lt;charT&gt;>
<font color="blue">class</font> basic_ios : <font color="blue">public</font> ios_base {
<font color="blue">public</font>:
    basic_ios(<font color="blue">const</font> basic_ios&) = <font color="blue">delete</font>;
    basic_ios& <font color="blue">operator</font>=(<font color="blue">const</font> basic_ios&) = <font color="blue">delete</font>;
};
</code></pre>

<pre><code>
<font color="blue">bool</font> isLucky(<font color="blue">int</font> number);              <font color="green">// we want only ints</font>
<font color="blue">bool</font> isLucky(<font color="blue">char</font>) = <font color="blue">delete</font>;           <font color="green">// reject chars</font>
<font color="blue">bool</font> isLucky(<font color="blue">bool</font>) = <font color="blue">delete</font>;           <font color="green">// reject bools</font>
<font color="blue">bool</font> isLucky(<font color="blue">double</font>) = <font color="blue">delete</font>;         <font color="green">// reject doubles and floats</font>
                                       <font color="green">// C++ prefers the conversion to double</font>
</code></pre>

<pre><code>
<font color="blue">template</font><<font color="blue">typename</font> T> <font color="blue">void</font> processPointer(<font color="blue">T</font>* ptr);

<font color="blue">template</font><> <font color="blue">void</font> processPointer<<font color="teal">void</font>>(<font color="blue">void</font>*) = <font color="blue">delete</font>;
<font color="blue">template</font><> <font color="blue">void</font> processPointer<<font color="teal">char</font>>(<font color="blue">char</font>*) = <font color="blue">delete</font>;
<font color="blue">template</font><> <font color="blue">void</font> processPointer<<font color="teal">const void</font>>(<font color="blue">const void</font>*) = <font color="blue">delete</font>;
<font color="blue">template</font><> <font color="blue">void</font> processPointer<<font color="teal">const char</font>>(<font color="blue">const char</font>*) = <font color="blue">delete</font>;
<font color="blue">template</font><> <font color="blue">void</font> processPointer<<font color="teal">const volatile void</font>>(<font color="blue">const volatile void</font>*) = <font color="blue">delete</font>;
<font color="blue">template</font><> <font color="blue">void</font> processPointer<<font color="teal">const volatile char</font>>(<font color="blue">const volatile char</font>*) = <font color="blue">delete</font>;
<font color="blue">template</font><> <font color="blue">void</font> processPointer<<font color="teal">std::wchar_t</font>>(std::wchar_t*) = <font color="blue">delete</font>;
<font color="blue">template</font><> <font color="blue">void</font> processPointer<<font color="teal">std::char16_t</font>>(std::char16_t*) = <font color="blue">delete</font>;
<font color="blue">template</font><> <font color="blue">void</font> processPointer<<font color="teal">std::char32_t</font>>(std::char32_t*) = <font color="blue">delete</font>;
<font color="blue">template</font><> <font color="blue">void</font> processPointer<<font color="teal">std::char8_t</font>>(std::char8_t*) = <font color="blue">delete</font>;
<font color="green">// void*, char* (cv-qualified or not)
// std::wchar_t (depending on compiler and target platform, for wide characters)
// std::char16_t (since C++11, UTF-16, 2 bytes, as std::uint_least16_t)
// std::char32_t (since C++11, UTF-32, 4 bytes, as std::uint_least32_t)
// std::char8_t (since C++20, UTF-8, 1 byte, as unsigned char, char, signed char)</font>
</code></pre>

<hr>

## OVERRIDED FUNCTIONS

Requirements for overriding class member functions:
* base class function must be `virtual`
* base and derived function names must match (except destructors)
* base and derived function parameter types must match
* base and derived function constness must match
* base and derived function return type and exception specification must be compatible
* base and derived function `reference qualifiers` must be identical (C++11)

**Member function reference qualifiers** limits use of a member function to `lvalues` or `rvalues` only. They need not be `virtual` to use them.

<pre><code>
<font color="blue">class</font> Widget {
<font color="blue">public</font>:
    <font color="blue">void</font> doWork() <font color="red">&</font>;                 <font color="green">// only when *this is an lvalue</font>
    <font color="blue">void</font> doWork() <font color="red">&&</font>;                <font color="green">// only when *this is an rvalue</font>
};

<font color="teal">Widget</font> w;
w.doWork();                          <font color="green">// calls Widget::doWork() &</font>
<font color="teal">Widget</font> makeWidget();
makeWidget().doWork();               <font color="green">// calls Widget::doWork() &&</font>
</code></pre>

Contextual keyword **override**:
* make explicit that a derived class function is supposed to override a base class version
* compiler will kvetch about all the overriding-related problems

Contextual keyword **final**:
* *(applied to a function)* prevents function from being overridden in derived class
* *(applied to a class)* prohibits from being used as a base class

<pre><code>
<font color="blue">class</font> Base {
<font color="blue">public</font>:
    <font color="blue">virtual void</font> mf1() <font color="blue">const</font>;
    <font color="blue">virtual void</font> mf2(<font color="blue">int</font> x);
    <font color="blue">virtual void</font> mf3() <font color="blue">&</font>;
    <font color="blue">virtual void</font> mf4() <font color="blue">const</font>;
    <font color="blue">void</font> mf5() <font color="blue">final</font>;
};

<font color="blue">class</font> Derived : <font color="blue">public</font> <font color="teal">Base</font> {
<font color="blue">public</font>:
    <font color="blue">virtual void</font> mf1() <font color="blue">const override</font>;
    <font color="blue">virtual void</font> mf2(<font color="blue">int</font> x) <font color="blue">override</font>;
    <font color="blue">virtual void</font> mf3() <font color="blue">& override</font>;
    <font color="blue">void</font> mf4() <font color="blue">const override</font>;
};
</code></pre>

<hr>

## UNCONDITIONAL NOEXCEPT SPECIFIER
* C++98-style exception remain valid, but they're deprecated (specifying exception types)
* it permits compilers to generate better object code (most optimizable function variant)
    * optimizers need not keep the runtime stack in an `unwindable state` if an exception would propagate out of the function, nor must they ensure that objects in a `noexcept` function are destroyed in the inverse order of construction should an exception leave the function
    * this is true only for `noexcept` specifier
* swapping higher-level data structures can generally be `noexcept` only if swapping their lower-level constituents is `noexcept`
* if an exception tries to leave the function in a `noexcept` function, the program will be terminated
* most functions are `exception-neutral`, that is they throw no exceptions themselves, but functions they call might emit one
    * exception-neutral functions are never `noexcept`, because they may emit 'just passing through' exceptions
* in C++11 all memory deallocation functions and all destructors, both user-defined and compiler-generated, are implicitly `noexcept`
    * the destructor is `noexcept` only if a data member of the class is of a type which specifies `'noexcept(false)'`


<pre><code>
<font color="blue">int</font> f(<font color="blue">int</font> x) <font color="blue">throw</font>();       <font color="green">// C++98 style; call stack unwounded + actions + termination</font>
<font color="blue">int</font> f(<font color="blue">int</font> x) <font color="blue">noexcept</font>;      <font color="green">// C++11 style; call stack is possibly unwounded + termination</font>
</code></pre>

<pre><code>
<font color="blue">template</font><<font color="blue">class</font> T, <font color="blue">size_t</font> N>
<font color="blue">void</font> swap(<font color="teal">T</font> (&a)[N], <font color="teal">T</font> (&b)[N]) <font color="blue">noexcept</font>(<font color="blue">noexcept</font>(swap(*a, *b)));

<font color="blue">template</font><<font color="blue">class</font> T1, <font color="blue">class</font> T2>
<font color="blue">struct</font> pair {
    <font color="blue">void</font> swap(<font color="teal">pair</font>& p) <font color="blue">noexcept</font>(<font color="blue">noexcept</font>(swap(first, p.first)), <font color="blue">noexcept</font>(swap(second, p.second)));
}
<font color="green">// STL's std::swap for std::pair</font>
</code></pre>

**functions with wide contracts**: never exhibits undefined behaviors, noexcept-friendly

**functions with narrow contracts**: if a precondition is violated, results are undefined


<hr>

## CONSTEXPR OBJECTS AND FUNCTIONS
* conceptually, `constexpr` indicates a value that's not only constant, it's known during compilation
* `constexpr` proclaims 'I can be used in a context where C++ requires a constant expression'
* by using `constexpr` whenever possible, you maximize the range of situations in which your objects and functions may be used
* when applied to **objects**, it's essentially a beefed-up form of `const`
    * `constexpr` objects values are determined during translation (compilation + linking)
    * values known during compilation may be placed in read-only memory (embedded systems)
    * `constexpr` values can be used in contexts where C++ requires an integral constant expression (array sizes, integral template arguments, enumerator values, alignment specifiers, ...)
    * all `constexpr` objects are `const`, but not all `const` objects are `constexpr`
* when applied to **functions**, things are more nuanced
    * the results of `constexpr` functions may not be `const`, nor their values are known during compilation
    * `constexpr` functions produce compile-time constants when they are called with compile-time constants
* I/O statements are generally not permitted in `constexpr` functions
    * if any of the arguments' values is not known during compilation, your code will be rejected (in contexts that demand compile-time constants)
    * `constexpr` function called with one or more values that are not known during compilation, it acts like a normal function, computing its result at runtime
    * restrictions imposed upon `constexpr` functions implementation differ between C++11 and C++14
    * in C++11, `constexpr` functions may contain no more than a single executable return statement, yet `ternary operator '?:'` and recursion are allowed
    * in C++11, `constexpr` member function cannot modify the object, they are implicitly `const`, they have `void` return types which isn't a literal type
    * in C++14, `constexpr` functions are limited to taking and returning literal types, which essentially means types that can have values determined during compilation (all built-in types except `void`, user-defined class with `constexpr` constructor)
    

<pre><code>
<font color="blue">int</font> sz;
<font color="blue">const auto</font> arraySize = sz;
<font color="teal">std::array</font><<font color="blue">int</font>, arraySize> data;        <font color="green">// ERROR, arraySize not known during compilation</font>
<font color="blue">constexpr auto</font> arraySize2 = 10;
<font color="teal">std::array</font><<font color="blue">int</font>, arraySize2> data2;      <font color="green">// OK, arraySize2 is constexpr</font>
</code></pre>

<pre><code>
<font color="blue">constexpr int</font> pow(<font color="blue">int</font> base, <font color="blue">int</font> exp) <font color="blue">noexcept</font> {
    <font color="blue">return</font> (exp == 0 <font color="red">?</font> 1 <font color="red">:</font> base * pow(base, exp - 1));
} <font color="green">// C++11</font>

<font color="blue">constexpr int</font> pow(<font color="blue">int</font> base, <font color="blue">int</font> exp) <font color="blue">noexcept</font> {
    <font color="blue">auto</font> result = 1;
    <font color="blue">for</font> (<font color="blue">int</font> i=0; i&lt;exp; ++i) result *= base;
    <font color="blue">return</font> result;
} <font color="green">// C++14</font>
</code></pre>

<pre><code>
<font color="blue">class</font> Point {
<font color="blue">public</font>:
    <font color="blue">constexpr</font> Point(<font color="blue">double</font> xVal = 0, <font color="blue">double</font> yVal = 0) <font color="blue">noexcept</font> : x(xVal), y(yVal) {}
    <font color="blue">constexpr double</font> xValue() <font color="blue">const noexcept</font> { <font color="blue">return</font> x; }
    <font color="blue">constexpr double</font> yValue() <font color="blue">const noexcept</font> { <font color="blue">return</font> y }
    <font color="blue">constexpr void</font> setX(<font color="blue">double</font> newX) <font color="blue">noexcept</font> { x = newX; }
    <font color="blue">constexpr void</font> setY(<font color="blue">double</font> newY) <font color="blue">noexcept</font> { y = newY; }
    
<font color="blue">private</font>:
    <font color="blue">double</font> x, y;
}; <font color="green">// constexpr setters are allowed in C++14</font>

<font color="blue">constexpr</font> <font color="teal">Point</font> midpoint(<font color="blue">const</font> <font color="teal">Point</font>& p1, <font color="blue">const</font> <font color="teal">Point</font>& p2) <font color="blue">noexcept</font> {
    <font color="blue">return</font> { (p1.xValue() + p2.xValue()) / 2, (p1.yValue() + p2.yValue()) / 2 };
} <font color="green">// non-member function</font>

<font color="blue">constexpr</font> <font color="teal">Point</font> reflection(<font color="blue">const</font> <font color="teal">Point</font>& p) <font color="blue">noexcept</font> {
    <font color="teal">Point</font> result;
    result.setX(-p.xValue());
    result.setY(-p.yValue());
    <font color="blue">return</font> result;
}

<font color="blue">constexpr</font> <font color="teal">Point</font> p1(9.4, 27.7);
<font color="blue">constexpr</font> <font color="teal">Point</font> p2(28.8, 5.3);
<font color="blue">constexpr auto</font> mid = midpoint(p1, p2);             <font color="green">// compile-time constants</font>
<font color="blue">constexpr auto</font> reflectedMid = reflection(mid);     <font color="green">// (-19.1, -16.5) known during compilation</font>
</code></pre>

<hr>

## CONST MEMBER FUNCTIONS
* **mutable**: permits modification of the class member declared `mutable` even if the containing object is declared `const`; `mutable` is used to specify that the member does not affect the external visible state of the `class` (mutexes, memo caches, lazy evaluation, ...)
* the safe bet is that `const` member functions will be a subject to concurrent execution, and that's why you should ensure that your `const` member functions are thread safe
* only data that are `mutable & shared` requires synchronization
    * unshared & mutable, unshared & immutable, shared & immutable are not the problem

<pre><code>
<font color="blue">class</font> Polynomial {
<font color="blue">public</font>:
    <font color="blue">using</font> RootsType = <font color="teal">std::vector</font><<font color="blue">double</font>>;
    
    <font color="teal">RootsType</font> roots() <font color="blue">const</font> {
        <font color="blue">if</font> (!rootsAreValid) { cacheRoots(); rootsAreValid = <font color="blue">true</font>; }
        return rootVals;
    }

<font color="blue">private</font>:
    <font color="blue">mutable bool</font> rootsAreValid{ <font color="blue">false</font> };
    <font color="blue">mutable</font> RootsType rootVals{};
};

<font color="teal">Polynomial</font> p;
<font color="blue">auto</font> rootsOfP = p.roots();                      <font color="green">// thread 1</font>
<font color="blue">auto</font> valsGivingZero = p.roots();                <font color="green">// thread 2
// data race: reading and writing the same memory without synchronization
// this code has undefined behavior
// the easiest way to address the issue is to employ a mutex
// mutable std::mutex m; as a field & std::lock_guard&lt;std::mutex&gt; g(m); in roots()
// locking and unlocking are non-const operations, thus mutable keyword (also applying M&M rule)
// std::mutex is a move-only type, thus Polynomial with m loses ability to be copy constructed</font>
</code></pre>

<pre><code>
<font color="blue">class</font> Point {
<font color="blue">public</font>:
    <font color="blue">double</font> distanceFromOrigin() <font color="blue">const noexcept</font> {
        ++callCount;
        <font color="blue">return</font> <font color="teal">std::sqrt</font>((x * x) + (y * y));
    }
    
<font color="blue">private</font>:
    <font color="blue">mutable</font> <font color="teal">std::atomic</font><<font color="blue">unsigned</font>> callCount{ 0 };
    <font color="blue">double</font> x, y;
};
<font color="green">// counting how many times a member function is called
// std::atomics are often a less expensive way to go than std::mutexes</font>
</code></pre>

<hr>

## SPECIAL MEMBER FUNCTION GENERATION
* in official C++ parlance, the special member functions are the ones that C++ is willing to generate on its own
    * these functions are generated only if they're needed, i.e., if some code uses them without their being expressly declared in the class
* C++98 has four such functions: the `default constructor`, the `destructor`, the `copy constructor`, the `copy assignment operator`
* C++11 has six such functions: the `default constructor`, the `destructor`, the `copy constructor`, the `copy assignment operator`, the `move constructor`, the `move assignment operator`
    * generated special member functions are implicitly `public` and `inline`
    * a `default constructor` is generated only if the class declares no constructor at all
    * a `destructor` is `noexcept` by default, and is `virtual` only if a base class destructor is `virtual`
    * a `copy constructor` is generated only if the class lacks a user-declared copy constructor
        * deleted if the class declares a move operation
        * won't be generated in a class with a user-declared copy assignment operator or destructor
    * a `copy assignment operator` is generated only if the class lacks a user-declared copy assignment operator
        * deleted if the class declares a move operation
        * won't be generated in a class with a user-declared copy constructor or destructor
    * a `move constructor` and a `move assignment operator` is generated only if the class contains no user-declared copy operations, move operations, or destructor


* the `move operations` perform 'memberwise moves' on the non-static data members of the class
    * they also move-construct its base class parts if they are any
    * the heart of each 'memberwise' move is application of `std::move`, that is a member will be moved if it is `move-enabled` (if it supports move semantics), otherwise it'll be copied
* the `move operations` are generated when they're needed
    * the move operations aren't generated if you declare them yourself
* copy operations are `independent`: copy constructor and copy assignment operator comes together (when both copy operations are treated as defaulted)
* move operations are `dependent`: if you declare either, that prevents compilers from generating the other
    * move operations won't be generated for any class that explicitly declares a copy operation
    * the rationale is that if you need a custom move constructor, then there's probably something wrong about generated memberwise move assignment operator
    * declaring a copy operation declares that the normal approach to memberwise copying an object isn't appropriate for the class
        * declaring a move operation causes compilers to disable the copy operations by deleting them
    * all of this is logical if you think about it, yet this is also about C++98-styled code to behave like in C++98
    

**The Rule of Three** states that if you declare any of a `copy constructor`, `copy assignment operator`, or `destructor`, you should declare all three. The justification for this is that almost always copy operation stemmed from the class performing some kind of resource management, which implied that:
* whathever resource management was being done in one copy operation probably needed to be done in the other copy operation
* the class destructor would also be participating in management of the resource, usually releasing it

A consequence of the `Rule of Three` is that the presence of a `user-declared destructor` indicates that simple memberwise copy is unlikely to be appropriate for the copying operations in the class. The existence of a user-declared destructor has no impact on compilers' willingness to generate copy operations. That continues to be the case in C++11, but only because restricting the conditions would break too much legacy code (all STL containers declare 'the big three').

So move operations are generated for classes (when needed) only if these three things are true:
* no `copy operations` are declared in the class
* no `move operations` are declared in the class
* no `destructor` is declared in the class


<pre><code>
<font color="blue">class</font> StringTable {
<font color="blue">public</font>:
    StringTable() {}
    
<font color="blue">private</font>:
    <font color="teal">std::map</font><<font color="blue">int</font>, <font color="teal">std::string</font>> values;
};
<font color="green">// copy & move operations and destructor will automatically be generated if they'll be used
// no user-declared copy operations, thus they'll be implicitly generated
// no explicit copy & move operations and destructor allows for generated move operations</font>

<font color="blue">class</font> StringTable {
<font color="blue">public</font>:
    StringTable() { makeLogEntry(<font color="purple">"Creating StringTable object."</font>); }
    ~StringTable() { makeLogEntry(<font color="purple">"Destroying StringTable object."</font>); }

<font color="blue">private</font>:
    <font color="teal">std::map</font><<font color="blue">int</font>, <font color="teal">std::string</font>> values;
};
<font color="green">// destructor side effect: it prevents the move operations from being generated
// the class is no longer move-enabled, but move requests will compile and run (as copies)</font>

<font color="blue">class</font> StringTable {
<font color="blue">public</font>:
    StringTable() { makeLogEntry(<font color="purple">"Creating StringTable object."</font>); }
    ~StringTable() { makeLogEntry(<font color="purple">"Destroying StringTable object."</font>); };
    StringTable(<font color="teal">StringTable</font>&&) = <font color="blue">default</font>;
    StringTable(<font color="blue">const</font> <font color="teal">StringTable</font>&) = <font color="blue">default</font>;
    StringTable& <font color="blue">operator</font>=(<font color="teal">StringTable</font>&&) = <font color="blue">default</font>;
    StringTable& <font color="blue">operator</font>=(<font color="blue">const</font> <font color="teal">StringTable</font>&) = <font color="blue">default</font>;
};
<font color="green">// no previous problems
// defaulted default constructor: implicitly defined even if other constructors are present</font>
</code></pre>

<pre><code>
<font color="blue">class</font> Widget {
    <font color="blue">template</font><<font color="blue">typename</font> T> Widget(<font color="blue">const T</font>& rhs);
    <font color="blue">template</font><<font color="blue">typename</font> T> Widget& <font color="blue">operator</font>=(<font color="blue">const T</font>& rhs);
};
<font color="green">// member function template: construct/assign Widget from anything
// compilers will still generate copy and move operations for Widget beside this templates</font>
</code></pre>

<pre><code>
<font color="blue">class</font> Base {
<font color="blue">public</font>:
    <font color="blue">explicit</font> Base(<font color="blue">int</font>) {}
    <font color="blue">virtual</font> ~Base() <font color="blue">noexcept = default</font>;              <font color="green">// virtual for polymorphic base class</font>
    Base(<font color="teal">Base</font>&&) = <font color="blue">default</font>;                          <font color="green">// moving</font>
    Base& <font color="blue">operator</font>=(<font color="teal">Base</font>&&) = <font color="blue">default</font>;               <font color="green">// moving at assignment</font>
    Base(<font color="blue">const</font> Base&) = <font color="blue">default</font>;                     <font color="green">// copying</font>
    Base& <font color="blue">operator</font>=(<font color="blue">const</font> <font color="teal">Base</font>&) = <font color="blue">default</font>;          <font color="green">// copying at assignment</font>
};
<font color="green">// The Rule of Five</font>
</code></pre>

A constructor with a single non-default parameter (until C++11) that is declared without the function specifier `explicit` is called a `converting constructor`. It is said that a converting constructor specifies an implicit conversion from the types of its arguments (if any) to the type of its class. Implicitly-declared and user-defined non-explicit `copy constructors` and `move constructors` are converting constructors.

<hr>

## STD::UNIQUE_PTR
* it's the same as `std::auto_ptr` but with move semantics that is unavailable in C++98
* by default, `std::unique_ptr's` are the same size as raw pointers (safe assumption)
* `std::unique_ptr` embodies exclusive ownership semantics
* a non-null `std::unique_ptr` always owns what it points to
* copying a `std::unique_ptr` isn't allowed, because it is a `move-only type`
* the common use cases for `std::unique_ptr's` are factory functions and Pimpl Idiom implementation
* there is no indexing `operator[]` for the single-object form `std::unique_ptr<T>`, while the array form `std::unique_ptr<T[]>` lacks dereferencing operators (`operator*` and `operator->`)
    * `std::unique_ptr<T[]>` make sense when you're using a C-like API that returns a raw pointer to a heap array that you assume ownership of
    * `std::array`, `std::vector`, `std::string` are virtually always better data structures choices than raw pointers
* if `std::unique_ptr` is a data member and object is being destroyed, `std::unique_ptr` would be also destroyed
    * if the ownership chain got interrupted due to an exception or other atypical control flow, the `std::unique_ptr` owning the managed resource would eventually have its destructor called
    * if an exception propagates out of a thread's primary function (for the program's initial thread) or if a `noexcept` specification is violated, local objects may not be destroyed
    * if `std::abort` or an exit function (`std::exit`, `std::quick_exit`, `std::_Exit`) is called, local objects definitely won't be destroyed
* by default, `std::unique_ptr` destruction would take place via `delete`, but, during construction, `std::unique_ptr` objects can be configured to use `custom deleters` (arbitrary functions, function objects, function pointers)
    * custom deleters may change the size of the smart pointer
        * deleters that are `function pointers` generally cause the size of a `std::unique_ptr` to grow from one word to two
        * for `function object` deleters the change in size is the sizeof() that function object
        * `stateless function objects` (from `lambda expressions` with no captures) incur no size penalty
    * when a `custom deleter` is to be used, its type must be specified as the second type argument to `std::unique_ptr`
        * deleter type is part of a pointer type
    * for custom deleters objects will ultimately be deleted via a base class pointer, thus, for that to work, the base class must have a `virtual` destructor
* C++11's smart pointers prohibit implicit conversions from a raw to smart pointer
    * to assume ownership of the object created via `new` we are using `reset()`
    * in templates, each use of `new` should be perfect-forwarded with `std::forward` for passed arguments to the function template
* `std::unique_ptr` easily and efficiently converts to a `std::shared_ptr`


<pre><code>
<font color="blue">template</font><<font color="blue">typename</font>... Ts> <font color="blue">auto</font> makeInvestment(Ts&&... params) {
    <font color="blue">auto</font> delInvmt = [](<font color="teal">Investment</font>* pInvestment) {
        makeLogEntry(pInvestment);
        <font color="blue">delete</font> pInvestment;
    };
    
    <font color="teal">std::unique_ptr</font><<font color="teal">Investment</font>, <font color="blue">decltype</font>(delInvmt)> pInv(<font color="blue">nullptr</font>, delInvmt);
    <font color="blue">if</font> (stock) { pInv.reset(<font color="blue">new</font> <font color="teal">Stock</font>(std::forward<Ts>(params)...)); }
    <font color="blue">else if</font> (bond) { pInv.reset(<font color="blue">new</font> <font color="teal">Bond</font>(std::forward<Ts>(params)...)); }
    <font color="blue">else if</font> (realEstate) { pInv.reset(<font color="blue">new</font> <font color="teal">RealEstate</font>(std::forward<Ts>(params)...)); }
    <font color="blue">return</font> pInv;
} 
<font color="green">// C++14; Stock & Bond & RealEstate : Investment</font>

<font color="blue">template</font><<font color="blue">typename</font>... Ts> 
<font color="teal">std::unique_ptr</font><<font color="teal">Investment</font>, <font color="blue">void</font> (*)(<font color="teal">Investment</font>*)> makeInvestment(Ts&&... params);
<font color="green">// sizeof(Investment) + sizeof(void (*)(Investment*))</font>

<font color="teal">std::shared_ptr</font><<font color="teal">Investment</font>> sp = makeInvestment(arguments);
<font color="green">// implicit conversion to std::shared_ptr</font>
</code></pre>

<hr>

## STD::SHARED_PTR
* an object accessed via `std::shared_ptr's` has its lifetime managed by those pointers through shared ownership
* no specific `std::shared_ptr` owns the object
    * instead, all `std::shared_ptr's` pointing to it collaborate to ensure its destruction at the point where it's no longer needed 
    * when the last `std::shared_ptr` pointing to an object stops pointing there, that `std::shared_ptr` destroys the object it points to
* a `std::shared_ptr` can tell whether it's the last one pointing to a resource by consulting the resource's `reference count`, a value associated with the resource that keeps track of how many `std::shared_ptr's` point to it
    * `std::shared_ptr` constructors increment this count (usually)
    * move construction and move assignment is faster than its copy equivalents, because it doesn't require incrementing the `reference count`
    * `std::shared_ptr` destructors decrement it
    * copy assignment operators do both
* the existence of the `reference count` has its performance implications
    * `std::shared_ptr's` are twice the size of a raw pointer
        * raw pointer to the resource + raw pointer to the resource's reference count
    * memory for the `reference count` must be dynamically allocated
        * pointed to objects have no place to store a reference count
        * cost of this may be avoided using `std::make_shared`
    * increments and decrements of the `reference count` must be `atomic` (to be thread-safe)
* `std::shared_ptr` uses `delete` as its default resource-destruction mechanism
* `std::shared_ptr` supports `custom deleters`
    * deleter type is not part of pointer type
    * specifying a custom deleter doesn't change the size of a `std::shared_ptr` object
    * regardless of deleter, a `std::shared_ptr` object is two pointers in size
    * custom deleters cannot be combined with `std::make_shared`
* the `reference count` is part of a larger data structure known as the `control block`
    * there's a control block for each object managed by `std::shared_ptr's`
    * control block may contains the reference count, a `weak count` (a secondary reference count), other data (custom deleters, custom allocators, ...)
    * thus, reinvisioned, `std::shared_ptr<T>` contains `PtrToT` and `PtrToControlBlock`
    * we assume that an object's control block is set up by the function creating the first `std::shared_ptr` to the object; in practice, we use rules for control block creation
        * `std::make_shared` always create a control block
        * a control block is created when a `std::shared_ptr` is constructed from a unique-ownership pointer
        * when a `std::shared_ptr` constructor is called with a raw pointer, it creates a control block
    * the usual control block implementation makes use of `inheritance` and `virtual function` to ensure that the pointed-to object is properly destroyed
* having dynamically allocated control block, arbitrarily large deleters and allocators, virtual function machinery, and atomic reference count manupulations, `std::shared_ptr` exact a very reasonable cost
    * the `control block` is only about three words in size, in typical conditions (defaulted)
    * its allocation is essentially free (it's incorporated into the memory allocation for the object being pointed to)
    * dereferencing is no more expensive than dereferencing a raw pointer
    * `atomic` operations entails to one-two individual machine instructions
    * the `virtual function machinery` in the control block is generally used only once per object at destruction time
* `std::shared_ptr's` are designed only for pointers to single objects, there's no `std::shared_ptr<T[]>`
    * you still can point to an array with `array-to-pointer decay` and specify a custom deleter with `delete[]`, yet `std::shared_ptr` offers no `operator[]`, so indexing requires expressions based on pointer arithmetic
    * declaring a smart pointer to a dumb array is almost always a sign of a bad design (use `std::vector` instead)
    

<pre><code>
sp1 = sp2;
<font color="green">// the reference count for sp1 is decremented, for sp2 is incremented</font>


<font color="blue">auto</font> loggingDel = [](<font color="teal">Widget</font> *pw) {
    makeLogEntry(pw);
    <font color="blue">delete</font> pw;
};

<font color="teal">std::unique_ptr</font><<font color="teal">Widget</font>, <font color="blue">decltype</font>(loggingDel)> upw(<font color="blue">new</font> <font color="teal">Widget</font>, loggingDel);
<font color="teal">std::shared_ptr</font><<font color="teal">Widget</font>> spw(<font color="blue">new</font> <font color="teal">Widget</font>, loggingDel);


<font color="blue">auto</font> pw = <font color="blue">new</font> <font color="teal">Widget</font>;                                  <font color="green">// raw pointer</font>
<font color="teal">std::shared_ptr</font><<font color="teal">Widget</font>> spw1(pw, loggingDel);          <font color="green">// create control block for *pw</font>
<font color="teal">std::shared_ptr</font><<font color="teal">Widget</font>> spw2(pw, loggingDel);          <font color="green">// create control block for *pw
                                                       // undefined behavior</font>
<font color="teal">std::shared_ptr</font><<font color="teal">Widget</font>> spw2(spw1);                    <font color="green">// same control block as spw1</font>
</code></pre>

<pre><code>
<font color="teal">std::vector</font><<font color="teal">std::shared_ptr</font><<font color="teal">Widget</font>>> processedWidgets;

<font color="blue">void</font> Widget::process() {
    processedWidgets.emplace_back(<font color="blue">this</font>);
    <font color="green">// it's passing a raw pointer to a container of std::shared_ptr's
    // thus they will create a new control block for the pointed-to Widget of *this
    // if there are std::shared_ptr's outside the member function that already point to that Widget,
    // it's match for undefined behavior</font>
}

<font color="blue">class</font> Widget : <font color="blue">public</font> std::enable_shared_from_this<<font color="teal">Widget</font>> { ... };
<font color="green">// if you want to be able to safely create a std::shared_ptr from a this pointer
// The Curiously Recurring Template Pattern (CRTP)
// a derived class inherits from a base class by templatizing on a derived class</font>

<font color="blue">void</font> Widget::process() {
    processedWidgets.emplace_back(shared_from_this());
    <font color="green">// add std::shared_ptr to current object to processedWidgets
    // std::enable_shared_from_this::shared_from_this is a member function
    // internally, it looks up the control block for the current object, and creates a referee
    // the design relies on the current object having an associated control block
    // so, there must an existing std::shared_ptr that points to the current object
    // if no such std::shared_ptr exists, behavior is undefined (but may throw an exception)</font>
}
</code></pre>

<hr>

## STD::WEAK_PTR
* `std::weak_ptr` tracks when it dangles, that is when the object it is supposed to point to no longer exists
    * `std::weak_ptr's` can detect when they dangle only when an object's lifetime is managed by `std::shared_ptr's`
* `std::weak_ptr's` are typically created from `std::shared_ptr's`, but they don't affect the `reference count` of the object they point to
* `std::weak_ptr` is an augmentation of `std::shared_ptr`
    * `std::weak_ptr` can't be dereferenced
    * `std::weak_ptr` can't be testes for nullness
* accessing `std::weak_ptr` may be done by creating a `std::shared_ptr` from the `std::weak_ptr`
* `std::weak_ptr's` objects are the same size as `std::shared_ptr's`, they make use of the same `control blocks`
* `std::weak_ptr's` don't participate in the shared ownership of objects
    * there's a second reference count in the control block, and it's manipulated by them (`weak count`)
* as long as `std::weak_ptr` refer to a control block, that control block must continue to exist

* as a use case for `std::weak_ptr` consider factory function that loads and caches expensive objects
    * a **second use case**: `Observer` design pattern
    * in most implementations, each `subject` contains a data member holding pointers to its `observers`
    * subjects have no interest in controlling the lifetime of their observers
    * subjects have a great interest in making sure that if an observer gets destroyed, subjects don't try to subsequently access it

    * a **third use case**: `A --(std::shared_ptr)--> B <--(std::shared_ptr)-- C`
    * we want to have a pointer from B back to A
    * there are three choices: a raw pointer, a `std::shared_ptr`, a `std::weak_ptr`
    * a raw pointer won't be able to detect a dangling pointer
    * a `std::shared_ptr` will provide a reference count of one for both A and B
    * a `std::weak_ptr` will be able to detect dangling pointer and will not interfere with the reference count

<pre><code>
<font color="blue">auto</font> spw = <font color="teal">std::make_shared</font><<font color="teal">Widget</font>>();
<font color="teal">std::weak_ptr</font><<font color="teal">Widget</font>> wpw(spw);
spw = <font color="blue">nullptr</font>;

<font color="blue">if</font> (wpw.expired()) {
    <font color="green">// std::weak_ptr's that dangle are said to have expired
    // note, that separating the check and the dereferencing action would introduce a race condition
    // another thread might be involved, thus causing an undefined behavior</font>
}

<font color="blue">auto</font> spw2 = wpw.lock();                         <font color="green">// null if wpw expired</font>
<font color="teal">std::shared_ptr</font><<font color="teal">Widget</font>> spw3(wpw);              <font color="green">// if wpw expired throws std::bad_weak_ptr</font>
</code></pre>

<pre><code>
<font color="teal">std::shared_ptr</font><<font color="blue">const</font> <font color="teal">Widget</font>> fastLoadWidget(<font color="teal">WidgetID</font> id) {
    <font color="blue">static</font> <font color="teal">std::unordered_map</font><<font color="teal">WidgetID</font>, <font color="teal">std::weak_ptr</font><<font color="blue">const</font> <font color="teal">Widget</font>>> cache;
    <font color="blue">auto</font> objPtr = cache[id].lock();
    
    <font color="blue">if</font> (!objPtr) {
        objPtr = loadWidget(id);
        cache[id] = objPtr;
    }
    
    <font color="blue">return</font> objPtr;
}
<font color="green">// a factory function that produces smart pointers to read-only objects based on a unique ID
// it's common for IDs to be used repeatedly, loadWidget() is an expensive call
// reasonable optimization is caching and destroying Widgets when they're no longer in use</font>
</code></pre>

<hr>

## STD::MAKE_UNIQUE & STD::MAKE_SHARED
* `std::make_shared` is part of C++11, `std::make_unique` is part of C++14
* STL `make functions` are: `std::make_shared`, `std::make_unique`, `std::allocate_shared`
    * make function take an arbitrary set of arguments, perfect-forwards them to the constructor for a dynamically allocated object, and return a smart pointer to that object
    * `std::allocate_shared` acts like `std::make_shared`, yet its first argument is an allocator object
* there are two reasons to prefer make functions
    * `make function` don't repeat type being created, `new` does
    * it provides exception safety
    * for `std::unique_ptr` custom deleters and braced initializers are scenerios where make functions are problematic
    * for `std::shared_ptr` problematic are custom deleters, braced initializers, custom operator `new`, custom operator `delete`
* `std::make_shared` improves code efficiency; this also applicable to `std::allocate_shared`
* make functions do not permit of `custom deleters`
* if the object type is quite large and the time between destruction of the last `std::shared_ptr` and the last `std::weak_ptr` is significant, a lag can occur between when an object is destroyed and when the memory it occupied is freed


<pre><code>
<font color="blue">auto</font> upw1(<font color="teal">std::make_unique</font><<font color="teal">Widget</font>>());
<font color="teal">std::unique_ptr</font><<font color="teal">Widget</font>> upw2(<font color="blue">new</font> Widget);

<font color="blue">void</font> processWidget(<font color="teal">std::shared_ptr</font><<font color="teal">Widget</font>> spw, <font color="blue">int</font> priority);
<font color="blue">int</font> computePriority();
processWidget(<font color="teal">std::shared_ptr</font><<font color="teal">Widget</font>>(<font color="blue">new</font> <font color="teal">Widget</font>), computePriority());
<font color="green">// potential resource leak
// compilers may emit code in order: new Widget, computePriority(), std::shared_ptr constructor</font>
processWidget(<font color="teal">std::make_shared</font><<font color="teal">Widget</font>>(), computePriority());
<font color="green">// if exception will be thrown, std::shared_ptr destructor will be invoked</font>

<font color="teal">std::shared_ptr</font><<font color="teal">Widget</font>> spw(<font color="blue">new</font> Widget);        <font color="green">// 2 allocations: Widget + control block</font>
<font color="blue">auto</font> spw = <font color="teal">std::make_shared</font><<font color="teal">Widget</font>>();          <font color="green">// 1 allocation: ditto but in single chunk</font>

<font color="blue">auto</font> widgetDeleter = [](<font color="teal">Widget</font>* pw) {};
<font color="teal">std::unique_ptr</font><<font color="teal">Widget</font>, <font color="blue">decltype</font>(widgetDeleter)> upw(<font color="blue">new</font> <font color="teal">Widget</font>, widgetDeleter);
<font color="teal">std::shared_ptr</font><<font color="teal">Widget</font>> spw(<font color="blue">new</font> <font color="teal">Widget</font>, widgetDeleter);
<font color="green">// cannot be done using make functions</font>
</code></pre>

<pre><code>
<font color="blue">auto</font> initList = { 10, 20 };
<font color="blue">auto</font> spv = <font color="teal">std::make_shared</font><<font color="teal">std::vector</font><<font color="blue">int</font>>>(initList);
<font color="green">// initList is a std::initializer_list object
// std::vector's perfect-forward code using parentheses, not braces
// if you want to construct pointed-to object using a braced initializers, you must use new directly
// braced initializers can't be perfect-forwarded
// using auto type deduction to create a std::initializer_list object is a workaround</font>
</code></pre>

<pre><code>
<font color="blue">class</font> ReallyBigType {};
<font color="blue">auto</font> pBigObj = <font color="teal">std::make_shared</font><<font color="teal">ReallyBigType</font>>();
<font color="green">// 1. Creating and working with std::shared_ptr's and std::weak_ptr's.
// 2. Last std::shared_ptr is being destroyed, but std::weak_ptr's remain.
// 3. Memory formerly occupied by large object remains allocated.
// 4. Last std::weak_ptr is being destroyed, memory for control block and object is released.</font>

<font color="teal">std::shared_ptr</font><<font color="teal">ReallyBigType</font>> pBigObj(<font color="blue">new</font> <font color="teal">ReallyBigType</font>);
<font color="green">// 1. As before.
// 2. As before, plus memory object is deallocated.
// 3. Only memory for the control block remains allocated.
// 4. Last std::weak_ptr is being destroyed, memory for control block is released.</font>
</code></pre>

<pre><code>
<font color="blue">void</font> processWidget(<font color="teal">std::shared_ptr</font><<font color="teal">Widget</font>> spw, <font color="blue">int</font> priority);
<font color="blue">void</font> cusDel(<font color="teal">Widget</font> *ptr);
processWidget(<font color="teal">std::shared_ptr</font><<font color="teal">Widget</font>>(<font color="blue">new</font> <font color="teal">Widget</font>, cusDel), computePriority());
<font color="green">// exception-unsafe call; we're passing an rvalues to processWidget()</font>

<font color="teal">std::shared_ptr</font><<font color="teal">Widget</font>> spw(<font color="blue">new</font> <font color="teal">Widget</font>, cusDel);
processWidget(spw, computePriority());
<font color="green">// when you use new directly, immediately pass the result to a smart pointer constructor...
// in a statement that does nothing else (compiler code emitting technique)
// if spw's constructor throws an exception, it's guaranteed that cusDel will be invoked
// we're passing an lvalue to processWidget()</font>

processWidget(std::move(spw), computePriority());
<font color="green">// optimization; moving std::shared_ptr requires no reference count manipulation</font>
</code></pre>

<hr>

## PIMPL IDIOM
**Pimpl Idiom**: a 'pointer to implementation' idiom
* that's the technique whereby you replace the data members of a class with a pointer to an implementation class
* used to combat excessive build times


<pre><code>
<font color="blue">class</font> Widget {                           <font color="green">// in header "widget.h"</font>
<font color="blue">public</font>:
    Widget();
    
<font color="blue">private</font>:
    <font color="teal">std::string</font> name;                    <font color="green">// in header &lt;string&gt;</font>
    <font color="teal">std::vector</font><<font color="blue">double</font>> data;            <font color="green">// in header &lt;vector&gt;</font>
    <font color="teal">Gadget</font> g1, g2, g3;                   <font color="green">// in header "gadget.h"</font>
};
<font color="green">// if a header's content changes, Widget clients must recompile</font>


<font color="blue">class</font> Widget {                           <font color="green">// in header "widget.h"</font>
<font color="blue">public</font>:
    Widget();
    ~Widget();                           <font color="green">// needed</font>
    
<font color="blue">private</font>:
    <font color="blue">struct</font> Impl;                         <font color="green">// just declared type (incomplete type)</font>
    <font color="teal">Impl</font> *pImpl;
};
<font color="green">// if something changes in headers, Widget clients are unaffected</font>


<font color="green">#include "widget.h"                      // "widget.cpp"
#include "gadget.h"
#include &lt;string&gt;
#include &lt;vector&gt;</font>

<font color="blue">struct</font> Widget::<font color="teal">Impl</font> {                    <font color="green">// define data members</font>
    <font color="teal">std::string</font> name;
    <font color="teal">std::vector</font><<font color="blue">double</font>> data;
    <font color="teal">Gadget</font> g1, g2, g3;
};

Widget::Widget() : pImpl(<font color="blue">new</font> <font color="teal">Impl</font>) {     <font color="green">// allocate data members</font>
}

Widget::~Widget() {
    <font color="blue">delete</font> pImpl;                        <font color="green">// destroy data members</font>
}
<font color="green">// C++98 style</font>
</code></pre>

<pre><code>
<font color="blue">class</font> Widget {                           <font color="green">// "widget.h"</font>
<font color="blue">public</font>:
    Widget();
<font color="blue">private</font>:
    <font color="blue">struct</font> Impl;
    <font color="teal">std::unique_ptr</font><<font color="teal">Impl</font>> pImpl;
};

<font color="blue">struct</font> Widget::<font color="teal">Impl</font> {                    <font color="green">// "widget.cpp"</font>
    <font color="teal">std::string</font> name;
    <font color="teal">std::vector</font><<font color="blue">double</font>> data;
    <font color="teal">Gadget</font> g1, g2, g3;
};

Widget::Widget() : pImpl(std::make_unique<Impl>()) {
}
<font color="green">// C++11 style</font>


<font color="green">#include "widget.h"</font>
<font color="teal">Widget</font> w;            <font color="green">// ERROR, applying sizeof/delete to an incomplete type!
                     // default deleter uses delete on a raw pointer
                     // default deleter employ static_assert to an incomplete type
                     // the type becomes complete when its definition has been seen</font>
</code></pre>                     

<pre><code>
<font color="blue">class</font> Widget {                                    <font color="green">// "widget.h"</font>
<font color="blue">public</font>:
    Widget();
    ~Widget();                                    <font color="green">// declaration only</font>
    Widget(<font color="teal">Widget</font>&& rhs);                         <font color="green">// declaration only</font>
    Widget& <font color="blue">operator</font>=(<font color="teal">Widget</font>&& rhs);              <font color="green">// declaration only</font>
    Widget(<font color="blue">const</font> <font color="teal">Widget</font>& rhs);                    <font color="green">// declaration only</font>
    Widget& <font color="blue">operator</font>=(<font color="blue">const</font> <font color="teal">Widget</font>& rhs);         <font color="green">// declaration only</font>

<font color="blue">private</font>:
    <font color="blue">struct</font> Impl;
    <font color="teal">std::unique_ptr</font><<font color="teal">Impl</font>> pImpl;
};

<font color="blue">struct</font> Widget::<font color="teal">Impl</font> {                             <font color="green">// "widget.cpp"</font>
    <font color="teal">std::string</font> name;
    <font color="teal">std::vector</font><<font color="blue">double</font>> data;
    <font color="teal">Gadget</font> g1, g2, g3;
};

Widget::Widget() : pImpl(std::make_unique<<font color="teal">Impl</font>>()) {
}

Widget::~Widget() = <font color="blue">default</font>;

Widget::Widget(<font color="teal">Widget</font>&& rhs) = <font color="blue">default</font>;

Widget& Widget::<font color="blue">operator</font>=(<font color="teal">Widget</font>&& rhs) = <font color="blue">default</font>;

Widget::Widget(<font color="blue">const</font> <font color="teal">Widget</font>& rhs) : pImpl(std::make_unique<<font color="teal">Impl</font>>(*rhs.pIml)) {
}

Widget& Widget::<font color="blue">operator</font>=(<font color="blue">const</font> <font color="teal">Widget</font>& rhs) {
    *pImpl = *rhs.pImpl;
    <font color="blue">return</font> *<font color="blue">this</font>;
}
<font color="green">// applying move and copy support
// move assignment operator needs to destroy the Impl object before reassigning it
// consider w1 = w2; for already existing  Widget's w1 and w2
// in terms of a move ctor, if an exception arises inside the ctor, dtor will invoke
// copy operations calls compiler-generated Widget::Impl's copy operations</font>
</code></pre>

<pre><code>
<font color="blue">class</font> Widget {                          <font color="green">// "widget.h"</font>
<font color="blue">public</font>:
    Widget();                           <font color="green">// plus compiler-generated special functions</font>
    
<font color="blue">private</font>:
    <font color="blue">struct</font> Impl;
    <font color="teal">std::shared_ptr</font><<font color="teal">Impl</font>> pImpl;
};

<font color="teal">Widget</font> w1;
<font color="blue">auto</font> w2(std::move(w1));
w1 = std::move(w2);
<font color="green">// for std::shared_ptr, the type of the deleter is not part of the type of the smart pointer
// thus, pointed-to types need not be complete when compiler-generated special functions are employed</font>
</code></pre>

<hr>

## THINGS TO REMEMBER

**Item 1**
* During template type deduction, arguments that are references are treated as non-references, i.e., their reference-ness is ignored.
* When deducing types for universal reference parameters, `lvalue` arguments get special treatment.
* When deducing types for by-value parameters, `const` and/or `volatile` arguments are treated as non-`const` and non-`volatile`.
* During template type deduction, arguments that are array or function names decay to pointers, unless they're used to initialize references.

**Item 2**
* `auto` type deduction is usually the same as template type deduction, but `auto` type deduction assumes that a braced initializer represents a `std:initializer_list`, and template type deduction doesn't.
* `auto` in a function return type or a lambda parameter implies template type deduction, not `auto` type deduction.

**Item 3**
* `decltype` almost always yields the type of a variable or expression without any modifications.
* For `lvalue` expressions of type `T` other than names, `decltype` always reports a type of `T&`.
* C++14 supports `decltype(auto)`, which, like `auto`, deduces a type from its initializer, but it performs the type deduction using the `decltype` rules.

**Item 4**
* Deduced types can often be seen using IDE editors, compiler error messages, and the `Boost TypeIndex` library.
* The results of some tools may be neither helpful nor accurate, so an understanding of C++'s type deduction rules remains essential.

**Item 5**
* `auto` variables must be initialized, are generally immune to type mismatches that can lead to portability or efficiency problems, can ease the process of refactoring, and typically require less typing than variables with explicitly specified types.

**Item 6**
* 'Invisible' proxy types can cause `auto` to deduce the 'wrong' type for an initializing expression.
* The `explicitly typed initializer idiom` forces `auto` to deduce the type you want it to have.

**Item 7**
* `Braced initialization` is the most widely usable initialization syntax, it prevents narrowing conversions, and it's immune to C++'s `most vexing parse`.
* During constructor overload resolution, braced initializers are matched to `std::initializer_list` parameters if at all possible, even if other constructors offer seemingly better matches.
* An example of where the choice between parentheses and braces can make a significant difference is creating a `std::vector<numeric_type>` with two arguments.
* Choosing between parentheses and braces for object creation inside templates can be challenging.

**Item 8**
* Prefer `nullptr` to `0` and `NULL`.
* Avoid overloading on integral and pointer types.

**Item 9**
* `typedef's` don't support templatization, but `alias` declarations do.
* `Alias templates` avoid the '::type' suffix, and, in templates, the `'typename'` prefix often required to refer to `typedefs`.
* C++14 offers `alias` templates for all the C++11 `type traits` transformations.

**Item 10**
* C++98-style `enums` are now known as `unscoped enums`.
* Enumerators of `scoped enums` are visible only within the `enum`. They convert to other types only with a cast.
* Both scoped and unscoped enums support specification of the underlying type. The default underlying type for scoped enums is `int`. Unscoped enums have no default underlying type.
* Scoped enums may always be forward-declared. Unscoped enums may be forward-declared only if their declaration specifies an underlying type.

**Item 11**
* Prefer deleted functions to `private` undefined ones.
* Any function may be deleted, including non-member functions and template instantiations.

**Item 12**
* Declare overriding functions `override`.
* `Member function reference qualifiers` make it possible to treat `lvalue` and `rvalue` objects (`*this`) differently.

**Item 13**
* Prefer `const_iterators` to `iterators`.
* In maximally generic code, prefer non-member versions of `begin`, `end`, `rbegin`, etc., over their member function counterparts.

**Item 14**
* `noexcept` is part of function's interface, and that means that callers may depend on it.
* `noexcept` functions are more optimizable than non-`noexcept` functions.
* `noexcept` is particularly valuable for the move operations, swap, memory deallocation functions, and destructors.
* Most functions are `exception-neutral` rather than noexcept.

**Item 15**
* `constexpr objects` are `const` and are initialized with values known during compilation.
* `constexpr functions` can produce compile-time results when called with arguments whose values are known during compilation.
* `constexpr` objects and functions may be used in a wider range of contexts than non-`constexpr` objects and functions.
* `constexpr` is part of an object's or function's interface.

**Item 16**
* Make `const` member functions thread safe unless you're certain they'll never be used in a concurrent context.
* Use of `std::atomic` variables may offer better performance than a mutex, but they're suited for manipulation of only a single variable or memory location.

**Item 17**
* The special member functions are those compilers may generate on their own: default constructor, destructor, copy operations, and move operations.
* Move operations are generated only for classes lacking explicitly declared move operations, copy operations, and a destructor.
* The copy constructor is generated only for classes lacking an explicitly declared copy constructor, and it's deleted if a move operation is declared. The copy assignment operator is generated only for classes lacking an explicitly declared copy assignment operator, and it's deleted if a move operation is declared. Generation of the copy operations in classes with an explicitly declared destructor is deprecated.
* Member function templates never suppress generation of special member functions.

**Item 18**
* `std::unique_ptr` is a small, fast, move-only smart pointer for managing resources with exclusive-ownership semantics.
* By default, resource destruction takes place via `delete`, but `custom deleters` can be specified. Stateful deleters and function pointers as deleters increase the size of `std::unique_ptr` objects.
* Converting a `std::unique_ptr` to a `std::shared_ptr` is easy.

**Item 19**
* `std::shared_ptr's` offer convenience approaching that of garbage collection for the shared lifetime management of arbitrary resources.
* Compared to `std::unique_ptr`, `std::shared_ptr` objects are typically twice as big, incur overhead for `control blocks`, and require `atomic` `reference count` manipulations.
* Default resource destruction is via `delete`, but `custom deleters` are supported. The type of the deleter has no effect on the type of the `std::shared_ptr`.
* Avoid creating `std::shared_ptr's` from variables of raw pointer type.

**Item 20**
* Use `std::weak_ptr` for `std::shared_ptr`-like pointers that can dangle.
* Potential use cases for `std::weak_ptr` include caching, observer lists, and the prevention of `std::shared_ptr` cycles.

**Item 21**
* Compared to direct use of `new`, `make functions` eliminate source code duplication, improve exception safety, and, for `std::make_shared` and `std::allocate_shared`, generate code that's smaller and faster.
* Situations where use of make functions is inappropriate include the need to specify custom deleters and a desire to pass braced initializers.
* For `std::shared_ptr's`, additional situations where make functions may be ill-advised include (1) classes with custom memory management and (2) systems with memory concerns, very large objects, and `std::weak_ptr's` that outlive the corresponding `std::shared_ptr's`.

**Item 22**
* The `Pimpl Idiom` decreases build times by reducing compilation dependencies between class clients and class implementations.
* For `std::unique_ptr` _pImpl_ pointers, declare special member functions in the class header, but implement them in the implementation file. Do this even if the default function implementations are acceptable.
* The above advice applies to `std::unique_ptr`, but not to `std::shared_ptr`.


<hr>

In [ ]:
# !pip3 install -U notebook-as-pdf
!jupyter-nbconvert --to PDFviaHTML effective_modern_cpp.ipynb